In [3]:
import geemap
import ipywidgets as widgets
import numpy as np
import pandas as pd
import proplot as plot 
import matplotlib.pyplot as plt 
from ipygee import chart
from pandas.plotting import register_matplotlib_converters
import ee

/usr/local/lib/python3.7/dist-packages/proplot/__init__.py:71: ProplotWarning: Rebuilding font cache. This usually happens after installing or updating proplot.
  register_fonts(default=True)


In [4]:
Map = geemap.Map()

admin = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1");
maharashtra = admin.filter(ee.Filter.eq('ADM1_NAME', 'Maharashtra'))
geometry = maharashtra.geometry()

point = ee.Geometry.Point([80.3288, 23.5121])

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterDate('2019-01-01', '2019-12-31') \
    .sort('CLOUD_COVER') \
    .min() \
    .select('B[1-7]') \
    .clip(geometry)


vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 5)
Map.addLayer(image, vis_params, "Landsat-8")

In [5]:
modis = ee.ImageCollection('MODIS/006/MCD12Q1') \
              .filterDate('2016-01-01', '2019-01-01') \
              .select('LC_Type1') \
              .reduce(ee.Reducer.median())

points = modis.sample(**{
    'region': ee.Geometry.Rectangle([72.0000, 23.0000, 82.0000, 15.0000]),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)


5000
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [75.96063229451673, 21.19164441922813]}, 'id': '0', 'properties': {'LC_Type1_median': 12}}


In [6]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'LC_Type1_median'
input = image.select(bands)

# Overlay the points on the imagery to get training.
training = input.sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)


{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 282, 'B2': 377, 'B3': 638, 'B4': 548, 'B5': 2163, 'B6': 1395, 'B7': 885, 'LC_Type1_median': 12}}


In [7]:
result = image.select(bands).classify(trained)

In [8]:
landcover = ee.ImageCollection('MODIS/006/MCD12Q1') \
              .filterDate('2019-01-01', '2019-12-31') \
              .select('LC_Type1') \
              .reduce(ee.Reducer.mean()) \
              .clip(geometry)

LandCoverVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
}

Map.centerObject(point, 5)
Map.addLayer(landcover, LandCoverVis, 'MODIS Land Cover')

In [9]:
landsatcover = result.set('classification_class_values', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
landsatcover = landsatcover.set('classification_class_palette', ['1c0dff', '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4',   ])
Map.addLayer(landsatcover, {}, 'Land cover')

In [10]:
ee_class_table = """

Value	Color	Description
0	1c0dff	Water
1	05450a	Evergreen needleleaf forest
2	086a10	Evergreen broadleaf forest
3	54a708	Deciduous needleleaf forest
4	78d203	Deciduous broadleaf forest
5	009900	Mixed forest
6	c6b044	Closed shrublands
7	dcd159	Open shrublands
8	dade48	Woody savannas
9	fbff13	Savannas
10	b6ff05	Grasslands
11	27ff87	Permanent wetlands
12	c24f44	Croplands
13	a5a5a5	Urban and built-up
14	ff6d4c	Cropland/natural vegetation mosaic
15	69fff8	Snow and ice
16	f9ffa4	Barren or sparsely vegetated
254	ffffff	Unclassified

"""

legend_dict = geemap.legend_from_ee(ee_class_table)
Map.add_legend(legend_title="Land Cover Legend", legend_dict=legend_dict)

In [11]:
Map

Map(center=[23.512099999999997, 80.3288], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…